In [1]:
!pip -q install langchain langchain_community langchain-groq tiktoken chromadb huggingface_hub transformers sentence-transformers einops

In [2]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Setting up LangChain


In [3]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.llm import LLMChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from glob import glob
import os
import pandas as pd

## Initialize models and embeddings


In [4]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initializing GROQ chat with provided API key, model name, and settings
model = ChatGroq(
            groq_api_key=groq_api_key, model_name="llama3-70b-8192",
                         temperature=0.2)

## Prompt templates

In [5]:
prompt_template_questions = """
You are a question bank generator for a question answering chatbot app.
Using the provided text below which is a section of a student manual, you are to generate all the possible questions from the students, teachers, parents, and staff that can be asked and answered within the given context.

------------
{text}
------------

Instructions:
Provide a complete list of questions that can be generated based on the context above..
Make sure the answers are exactly from the text (in verbatim) and should not be replaced with any synonyms or added with extra words.
This is for training an extractive question answering model.
Make sure that no parts of the contexts are missed.
Generate at least 30 questions in which the order of the questions match the order of appearance in the context.
Do not include questions that cannot be answered using the context.
Do not repeat any questions or generate the same question again in the list of questions.

Response format for each item: Must have a Q: before the question and A: before the answer, like:
Q: Question
A: Answer

QUESTIONS AND ANSWERS:
"""

PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["text"],
    template=prompt_template_questions
)

## Load multiple and process documents

In [6]:
# Initialize documents and path
documents = []
root  = os.getcwd()

# Sort the file names
paths = sorted(glob('*.txt'))

# Copy each file to new file
for path in paths:
  loader = TextLoader(path)
  document = loader.load()
  documents.extend(document)

## Loop through each topic and generate questions

In [7]:
# Initialize values
topic = 0
QA_pairs = []
raw_responses = ''

for doc in documents:

  # Splitting the text into chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
  texts = text_splitter.split_documents([doc])

  # Initialize question generation chain
  question_gen_chain = LLMChain(llm=model, prompt=PROMPT_QUESTIONS)

  # Run question generation chain
  response = question_gen_chain.invoke(texts)

  # Reset count of QA pairs
  pair_count = 0

  # Segregate and store questions and answers
  sentences = response['text'].split('\n')
  for sentence in sentences:
    if sentence.startswith('Q:'):
      Q = sentence[3:]
    elif sentence.startswith('A:'):
      A = sentence[3:]
      QA_pairs.append([Q,A,topic])
      pair_count += 1

  # Append raw response to file
  raw_responses += f'Generated questions for topic {topic}:\n\n{response["text"]}\n\n'

  print(f'Topic {topic}: Generated {pair_count} pairs of questions')

  # Increment topic count
  topic += 1

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Topic 0: Generated 70 pairs of questions
Topic 1: Generated 38 pairs of questions
Topic 2: Generated 27 pairs of questions
Topic 3: Generated 31 pairs of questions
Topic 4: Generated 53 pairs of questions
Topic 5: Generated 34 pairs of questions
Topic 6: Generated 29 pairs of questions
Topic 7: Generated 17 pairs of questions
Topic 8: Generated 22 pairs of questions
Topic 9: Generated 21 pairs of questions
Topic 10: Generated 18 pairs of questions
Topic 11: Generated 24 pairs of questions
Topic 12: Generated 20 pairs of questions
Topic 13: Generated 29 pairs of questions
Topic 14: Generated 23 pairs of questions
Topic 15: Generated 22 pairs of questions
Topic 16: Generated 31 pairs of questions
Topic 17: Generated 23 pairs of questions
Topic 18: Generated 28 pairs of questions
Topic 19: Generated 32 pairs of questions
Topic 20: Generated 26 pairs of questions
Topic 21: Generated 26 pairs of questions
Topic 22: Generated 28 pairs of questions
Topic 23: Generated 29 pairs of questions
To

## Storing the QA pairs

In [8]:
# Create a DataFrame from the list of question-answer pairs
qa_df = pd.DataFrame(QA_pairs, columns=["Question", "Answer", "Topic"])

# Save the DataFrame to a CSV file
qa_df.to_csv("questions_and_answers.csv", index=False)
print('Successfully saved questions and answers to CSV!')

# Save raw responses to a text file
with open('raw_responses.txt','w') as output_file:
  output_file.write(raw_responses)

Successfully saved questions and answers to CSV!


## Displaying the QA pairs per topic

In [10]:
# Print sample questions per topic
pd.set_option('display.max_colwidth', None)  # or 199
for i in range(1, len(documents)+1):
  df_topic = qa_df[qa_df['Topic'] == i]
  display(df_topic.head(100))

,Question,Answer,Topic
70,What is the University of San Carlos (USC) administered by?,Society of the Divine Word (SVD) missionaries.,1
71,Since what year has the University of San Carlos (USC) been administered by the Society of the Divine Word (SVD) missionaries?,1935.,1
72,What type of educational institution is the University of San Carlos (USC)?,A Catholic educational institution.,1
73,What is offered by the University of San Carlos (USC) from kindergarten to graduate school?,The complete educational package.,1
74,What is the University of San Carlos (USC) also referred to as?,University.,1
75,In what year did the University of San Carlos (USC) become a University?,1948.,1
76,What prompted the expansion of the University to what was then called the Boys’ High School in 1956?,Rapid growth in the ‘50s saturated the campus near the city center.,1
77,What was the Boys’ High School later renamed to?,North Campus.,1
78,In what year did the University expand to the Teacher Education Center and Girls’ High School?,1964.,1
79,What was the Teacher Education Center and Girls’ High School later renamed to?,South Campus.,1


,Question,Answer,Topic
108,"What is embodied in the two statements ""Education with a Mission"" and ""Witness to the Word""?",The University's Catholic identity.,2
109,What is the aim of USC in the holistic formation of the individual?,"To enlighten the mind, hone skills, and positively transform character and behavior.",2
110,What is the goal of every Carolinian?,"To become a catalyst towards the realization of the kingdom of God, promoting truth, love, justice, freedom, peace, and grace.",2
111,How are Carolinians recognizable in society?,"By their contribution as the ""light"", ""salt"", and ""yeast"" of the Gospel.",2
112,What are the characteristics of graduates of USC?,"Professionally competent and skilled (Scientia), having noble character and being value-driven (Virtus), and dedicated to social transformation (Devotio).",2
113,What does the University of San Carlos envision for the world?,A world where the darkness of sin and the night of unbelief vanish before the light of the Word and the Spirit of grace.,2
114,What does the University of San Carlos envision for society?,"A society where citizens are competent, noble in character, and community-oriented.",2
115,What is the mission of the University of San Carlos?,"To develop competent and socially responsible professionals and lifelong learners in an environment that fosters excellence in the academic core processes of teaching-learning, research, and community extension service.",2
116,What is the goal of the University's mission?,"To provide timely, relevant, and transformative academic programs responsive to the needs of the local, national, and global communities in a rapidly changing world.",2
117,What are the University Core Values?,Integrity; Excellence; Commitment; Social responsibility; Evangelization; and Leadership.,2


,Question,Answer,Topic
135,What is the University seal?,The University seal is the official insignia of the University of San Carlos.,3
136,What does the University seal depict?,The University seal depicts a shield on either side by seven laurel leaves.,3
137,What do the laurel leaves stand for?,The laurel leaves stand for excellence.,3
138,How many gifts of the Holy Spirit do the laurel leaves symbolize?,"Each one symbolizes a gift of the Holy Spirit: wisdom, understanding, counsel, fortitude, knowledge, piety, and fear of the Lord.",3
139,How is the shield divided?,The shield is divided into three parts.,3
140,What is on the upper left of the shield?,On the upper left of the shield is a globe encircled and topped by a cross.,3
141,What does the globe encircled and topped by a cross symbolize?,This is the symbol of the SVD in the world.,3
142,What is on the upper right part of the shield?,On the upper right part of the shield is Magellan’s Cross.,3
143,What does Magellan’s Cross symbolize?,"Magellan’s Cross symbolizes Cebu City, where USC is located.",3
144,What is on the bottom half of the shield?,On the bottom half of the shield is an open book.,3


,Question,Answer,Topic
166,When was the Colegio de San Ildefonso established?,21 August 1595,4
167,Who established the Colegio de San Ildefonso?,"Jesuit priests Antonio Sedeño, Alonso de Humanes, Mateo Sanchez, and a lay brother Gaspar Garay",4
168,Why was the Colegio de San Ildefonso closed in 1769?,Because of the papal expulsion of the Jesuits from Spain and all Spanish possessions in 1767,4
169,Who took over the ownership of the colegio and its properties in 1779?,"The Bishop of Cebu, Msgr. Rubio de Arevalo",4
170,What was the colegio re-established as in 1779?,The Real Seminario de San Carlos,4
171,Who took over the colegio in 1852?,The Dominican priests,4
172,What was the colegio renamed to in 1852?,Seminario Conciliar de San Carlos,4
173,Who took over the colegio in 1867?,The Vincentians,4
174,What was the colegio renamed to in 1867?,Seminario-Colegio de San Carlos,4
175,When did the Seminario-Colegio de San Carlos confer the Bachelor’s degree to its first graduates?,1894,4


,Question,Answer,Topic
219,Who are the three university saints?,"San Carlos Borromeo (1538-1584), St. Arnold Janssen (1837-1909), St. Joseph Freinademetz (1852-1908).",5
220,Who owns and manages the University of San Carlos?,The Society of Divine Word (SVD).,5
221,When is the feast of St. Charles Borromeo?,November 4.,5
222,What was St. Charles Borromeo's role in the Council of Trent?,He facilitated the final deliberations of the Council of Trent and took a large share in the drafting of the Tridentine Catechism.,5
223,How did St. Charles Borromeo assist the poor during the plague?,"He bravely attended to the sick and the dead victims of the plague, sparing no expense and avoiding no danger in an effort to assist the poor.",5
224,What was St. Charles Borromeo's role in the reformation movement?,He played a major role against the reformation movement.,5
225,What was St. Charles Borromeo's contribution to education?,"He worked towards the reform of collegiate churches, seminaries, colleges, and communities for the education, especially of candidates for holy orders.",5
226,Why was St. Charles Borromeo adopted as the patron of the University of San Carlos?,Because he is the Patron of Seminaries.,5
227,Who founded the Society of Divine Word (SVD)?,St. Arnold Janssen.,5
228,Where was St. Arnold Janssen born?,"In the village of Goch in the Rhineland, Germany, not far from the Dutch border.",5


...

,Question,Answer,Topic
1668,What does the Athletics Office cater to?,"Queries and concerns regarding tryouts, privileges, booking of sports facilities, policies regarding sports events, rental rates of sports facilities such as gyms, swimming pools, and tennis court.",60
1669,How can the Athletic Office be contacted?,"Through landline (032) 2300100 local 523, or through the following persons and emails: Fr. Adriano Ocariza (acocariza@usc.edu.ph), Marvin Avergonzado (mnavergonzado@usc.edu.ph), Patrick Philipp Gamaliel Uno (pguno@usc.edu.ph).",60
1670,What does the Campus Ministry Office cater to?,"Queries regarding Liturgical Services, Prayer Intentions, Sacraments, Blessing, Spiritual Direction and Counselling, Vocation and Mission Animation, Faith Sharing, Retreat and Recollection Giving, Lector and Altar Servers Training, Basic Bible Seminar, JPIC, and Arnold Janssen Spirituality Workshop.",60
1671,Where is the Campus Ministry Office located?,"Under St. Arnold Janssen and St. Joseph Freinademetz Chapel in Talamban Campus, and 3rd Floor Dingman Building in Downtown Campus.",60
1672,How can the Campus Ministry Office be contacted?,"Through the Office of the University Chaplain (Landline: (032) 2300100 local 251, Email: univchaplain@usc.edu.ph), Talamban Campus Office (Landline: (032) 2300100 local 135, Email: univchaplain@usc.edu.ph), Downtown Campus Office (Landline: (032) 2300100 local 540/541, Email: chaplaindc@usc.edu.ph).",60
1673,What does the Community Extension Services (CES) Office cater to?,"Queries regarding CES activities, CES events, CES programs, CES projects, CES forms, CES application.",60
1674,What programs does the Community Extension Services (CES) Office offer?,"Mission Exposure Program, Research and Development Program, and CES Carolinian Volunteers program.",60
1675,How can the Community Extension Services (CES) office be contacted?,"Through landline (032) 2300100 local 549, or through its head, Bonifacio N. Mercado Jr. (bnmercado@usc.edu.ph, ces@usc.edu.ph).",60
1676,Where is the Community Extension Services (CES) office located?,"2nd floor of USC Mission Center, Fr. Ludwig Lehmeier Building, Downtown Campus.",60
1677,What does the Mission Exposure Program help students to do?,"Understand the importance of context in pursuing development, demonstrate devotion to work for the poor and those in the margins, define the contexts of varied communities where mission exposure is done, and describe the challenges confronting the members of the community.",60


,Question,Answer,Topic
1701,What does the Information Resource Management Office (IRMO) cater to?,"Queries regarding ISMIS, IT Helpdesk, Email, RFID, Canvas, USC Emails, and USC Google Accounts.",61
1702,Where is the IRMO located in Downtown Campus?,Basement Floor of Arthur Dingman Building.,61
1703,Where is the IRMO located in Talamban Campus?,3rd Floor of Lawrence Bunzel Building.,61
1704,How can the IRMO be contacted?,Through landline (032) 2300100 local 107/506 and email (irmoacs@usc.edu.ph).,61
1705,What does the USC Library cater to?,"Queries regarding Unreturned books, Unsettled library fines, Borrowing of books, Status of books, and Access to Online Resources and Services.",61
1706,Where is the USC Library located?,Josef Baumgartner Learning Resource Center (JB-LRC) of Talamban Campus.,61
1707,How can the USC Library be contacted?,"Through landline (032) 2300100 local 198 or through the following persons and emails: Sharleen G. Ramos (sgramos@usc.edu.ph), Divina Tenorio (dctenorio@usc.edu.ph), Janice Berdin (jfberdin@usc.edu.ph).",61
1708,"What does the Office of Alumni Affairs, Scholarships and Job Placement cater to?","Queries regarding Scholarships, USC Student Financial Assistance Program and Sponsors, Scholarship Application Form, Procedure and Requirements, Scholarship Confirmation, Release of Allowances for Scholars, Status of Previlege, Balance, Delay of Payment, Letter-Explanation for Deficiency in Grades, Employment Opportunities, Internship Opportunities, Partner Companies, Alumni matters, and Alumni ID for Fresh Graduates.",61
1709,"Who handles scholarships in the Office of Alumni Affairs, Scholarships and Job Placement?","Christine Y. Alpis (Landline: (032) 2300100 local 276/406, Email: scholarship@usc.edu.ph).",61
1710,"Who handles alumni affairs in the Office of Alumni Affairs, Scholarships and Job Placement?",Ferdinand S. Boncayao (Email: fsboncayao@usc.edu.ph and alumni@usc.edu.ph).,61


,Question,Answer,Topic
1752,What programs does the Department of Accountancy offer?,"BS Accountancy, BS Management Accounting, MS in Accountancy Major In Management Accounting.",62
1753,How can the Department of Accountancy be contacted?,"Through landline (032) 2300100 local 513, or through the following persons and emails - Chair: Wendell K. Ang, JD-ILP, CPA (wkang@usc.edu.ph), Assistant Chair: Mary Anne B. Jamisola-Adriano, JD, CPA (majamisola@usc.edu.ph), Secretary: Venus M. Branzuela (vmbranzuela@usc.edu.ph).",62
1754,Who is the Chair of the Department of Accountancy?,"Wendell K. Ang, JD-ILP, CPA.",62
1755,What is the email address of the Chair of the Department of Accountancy?,wkang@usc.edu.ph.,62
1756,"What programs does the Department of Anthropology, Sociology and History offer?","BA Anthropology, MA Anthropology, Certificate Course in Cebuano Heritage Studies and Conservation Management, and Certificate Course in Sustainable Community Development.",62
1757,"How can the Department of Anthropology, Sociology and History be contacted?","Through landline (032) 2300100 local 139, or through the following persons and emails - Chair: Dr. Leny G. Ocasiones (lgocasiones@usc.edu.ph), Secretary: Angela June N. Rosal (dashsec@usc.edu.ph).",62
1758,"Who is the Chair of the Department of Anthropology, Sociology and History?",Dr. Leny G. Ocasiones.,62
1759,"What is the email address of the Chair of the Department of Anthropology, Sociology and History?",lgocasiones@usc.edu.ph.,62
1760,What programs does the Department of Architecture offer?,"BS Architecture, BS Interior Design, Bachelor of Landscape Architecture, Master of Architecture (majors in Interior Architecture, Urban Design, Architectural Science, and Landscape Architecture), and Master in Urban Planning.",62
1761,How can the Department of Architecture be contacted?,"Through landline (032) 2300100 local 208, or through the following persons and emails - Chair: Ar. Adrian I. Yap (aiyap@usc.edu.ph), Secretary: Ms. Jessyl Pestaño (archdept@usc.edu.ph), Coordinator for Architecture Graduate Programs: Ar. Robert B. Malayao (rbmalayao@usc.edu.ph).",62


,Question,Answer,Topic
1812,What department does not offer any program but instead handles General Education courses and Education with a Mission (EDM) courses?,The Department of General Education,63
1813,How can the Department of General Education be contacted?,"Through landline (032) 2300100 local 317, or through the following persons and emails - Director: Grace Magalzo-Bualat, MPS, LLB (gmbualat@usc.edu.ph), Secretary: Charin R. Salindo (charinsalindoogem@gmail.com)",63
1814,"Who is the coordinator for EDM 1, EDM 2, and Ethics?","Fr. Jerome S. Montesclaros, PhD, SVD (jeromesvd@yahoo.com)",63
1815,"Who is the coordinator for TCW, PC, LWR, and RPH?","Owen O. de la Victoria, LLB, MAPA (oodelavictoria@usc.edu.ph)",63
1816,"Who is the coordinator for STS, UTS, MMW, and ART?","Val B. Salares, MS (vbsalares@usc.edu.ph)",63
1817,What programs are offered by the Department of Hospitality and Tourism?,"BS Hospitality Management, BS Tourism Management, and Diploma in Culinary Arts",63
1818,How can the Department of Hospitality and Tourism be contacted?,"Through landline (032) 2300100 local 555/556, or through the following persons and emails - Chair: Dr. Myrla Mercedes B. Gidayawan (mbgidayawan@usc.edu.ph), Assistant Chair: Mr. Demetrio S. Anticuando (dsanticuando@usc.edu.ph), Secretary: Ms. Shalymar C. Urbano (scurbano@usc.edu.ph)",63
1819,Who is the program head for BS Hospitality Management and Diploma in Culinary Arts?,Dr. Myrla Mercedes B. Gidayawan (mbgidayawan@usc.edu.ph),63
1820,Who is the program head for BS Tourism Management?,Mr. Demetrio S. Anticuando (dsanticuando@usc.edu.ph),63
1821,What programs are offered by the College of Law?,"Juris Doctor (non-thesis), Juris Doctor (with thesis)",63


,Question,Answer,Topic
